In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
# Define the data directories and image dimensions
data_dir = 'dataset'
img_width = 128
img_height = 128
batch_size = 32

In [ ]:
# Set the path to your data directory
data_dir = "C:/Users/DELL/Desktop/dataset/D-E"

# Get the list of all image file names
image_files = os.listdir(data_dir)

# Get the list of all pass image file names
pass_images = [file for file in image_files if "pass" in file]

# Get the list of all fail image file names
fail_images = [file for file in image_files if "fail" in file]

# Split the data into train and test sets (80% train, 20% test)
pass_train, pass_test = train_test_split(pass_images, test_size=0.2, random_state=42)
fail_train, fail_test = train_test_split(fail_images, test_size=0.2, random_state=42)

# Create the train and test directories
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Copy the pass images to the train and test directories
for file in pass_train:
    shutil.copy(os.path.join(data_dir, file), os.path.join(train_dir, file))
for file in pass_test:
    shutil.copy(os.path.join(data_dir, file), os.path.join(test_dir, file))

# Copy the fail images to the train and test directories
for file in fail_train:
    shutil.copy(os.path.join(data_dir, file), os.path.join(train_dir, file))
for file in fail_test:
    shutil.copy(os.path.join(data_dir, file), os.path.join(test_dir, file))

In [ ]:
# Define the CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add Dropout layer after last Convolutional layer
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Define the data generators for image augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load the training and testing data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)

In [ ]:
# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Train the model
history = model.fit(train_data, epochs=10, validation_data=test_data, metrics=['accuracy', 'loss', 'val_accuracy', 'val_loss'],callbacks=[early_stop])

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Save the trained model
model.save('my_model.h5')